In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
# input_path = "s3://thotadhanasekhar0001/CUSTOMERS/"

input_path = "/Volumes/my_catalog/raw_retailx/orders/*/" 
bronze_tbl = "my_catalog.bronze_retailx.orders"
checkpoint_base = "/Volumes/my_catalog/_system/checkpoint/retailx_orders/"

schema_path = checkpoint_base + "schema"
checkpoint_path = checkpoint_base + "checkpoint"

In [0]:
df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
    .option("cloudFiles.schemaLocation", schema_path)
    .load(input_path)
)

# .option("pathGlobaFilter", "*.csv")
# .option("cloudFiles.maxFilesPerTrigger", "1")
#.option("cloudFiles.maxBytesPerTrigger", "100000000")

In [0]:
df_bronze = (
    df
    .withColumn("_ingested_at", current_timestamp())
    .withColumn("_source_file", col("_metadata.file_path"))
    #.withColumn("_file", col("_metadata.file_name"))
    #.withColumn("_file_mod_time", col("_metadata.file_modification_time"))
)

In [0]:
(
    df_bronze.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path) 
    .trigger(availableNow=True) 
    .toTable(bronze_tbl)     
)

In [0]:
df_customers = spark.read.table(bronze_tbl)
display(df_customers)

ORDER_ID,CUSTOMER_ID,ORDER_DATE,AMOUNT,_rescued_data,_ingested_at,_source_file
101,1,2025-04-01,1200,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
102,1,2025-04-10,850,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
103,1,2025-05-02,2300,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
104,2,2025-04-03,1500,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
105,2,2025-04-18,950,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
106,2,2025-05-05,1800,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
107,3,2025-04-06,2200,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
108,3,2025-04-22,1100,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
109,3,2025-05-10,2750,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
110,4,2025-04-08,900,null,2025-12-14T05:37:54.125Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%201.csv
